# Working with ISIC Images
Images are at the heart of the ISIC Archive, and this notebook
will give you a sense of what the ```isicarchive``` package
can do in terms of retrieving images from the archive, and the
kinds of methods you can apply to them.
## Preparation
We'll start again by creating an ```IsicApi``` object, which
allows us to access the various pieces of information and images
stored in the archive. The call to ```cache_images()``` will
then download information (and meta information) about all
images that are available--if you are logged in as a regular
user, you may have access to additional images! **If you're
using the ```cache_images()``` method, please use a different
cache folder if you want to work with both publicly available
and private images in separate sessions.**

Next, we'll select all images from the ```IsicApi``` object that
have a specific diagnosis, and then display those images.

In [ ]:
# Settings (please adapt to your needs!)
cache_folder = 'C:\\Users\\weberj3\\Documents\\ISIC'

# Preparation
from isicarchive import IsicApi, func
api = IsicApi(cache_folder=cache_folder)
api.cache_images()

In [ ]:
import imageio
image = imageio.imread('C:\\Users\\weberj3\\Documents\\ISIC\\0\\image_59e509e4d831136981ee67a0_ISIC_0021137.jpg')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
imobj = plt.imshow(image)
imobj.axes.set_axis_off()
plt.show()
print(True)

In [ ]:
image = api.image('ISIC_0000000')
image.load_imagedata()
image.load_superpixels()
image.map_superpixels()
image_data = image.data
image_shape = image_data.shape
image_data.shape = (image_shape[0] * image_shape[1], -1)
map = image.superpixels['map']
superpixel_index = 472
pixel_count = map[superpixel_index, -1]
superpixel_pixels = map[superpixel_index, 0:pixel_count]
image_data[superpixel_pixels, 0] = 255
image_data[superpixel_pixels, 1] = 0
image_data[superpixel_pixels, 2] = 0
image_data.shape = image_shape

import matplotlib.pyplot as plt
plt.imshow(image_data)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image_data)
plt.show()

In [ ]:
from isicarchive import func
import imageio
import numpy
spimg = imageio.imread('C:\\Users\\weberj3\\Documents\\ISIC\\0\\spimg_57eea3d99fc3c12a89bb5e00.png')
spmap = func.superpixel_map_rgb(spimg)
image = func.color_superpixels(spimg.shape, [x for x in range(0,900)], spmap, [255,0,0], 1.0, [v for v in numpy.arange(0.0, 0.9,0.001)])
func.display_image(image)